## Анализ базы данных крупного сервиса для чтения книг

<b> Описание данных: </b>

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

<b> Цель проекта: </b> Проанализировать информацию о книгах, издательствах, авторах и пользовательских обзорах книг, для формирования ценностного приложения для читателей

<b>План работы: </b>  <a id="cell0"></a>

[1. Загрузка и знакомство с данными](#cell1)


2. SQL-запросы для решения заданий: 

   [2.1 Определение количества книг вышедших после 1 января 2000 года](#cell2)
   
   [2.2 Определение количества обзоров и средней оценки для каждой книги](#cell3)
   
   [2.3 Определение издательства, которое выпустило наибольшее число книг толще 50 страниц](#cell4)
   
   [2.4 Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками](#cell5)
   
   [2.5 Расчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок](#cell6)
   
   
[3. Общий вывод](#cell7)


## 1. Загрузка и знакомство с данными   <a id="cell1"></a>  <font size = 2>[к оглавлению](#cell0)</font>

In [1]:
import pandas as pd
#from sqlalchemy import create_engine
from sqlalchemy import create_engine, text

Доступ к базе данных

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})



Выведем первые строки всех исследуемых таблиц

In [3]:
def result(table): # создадим функцию для запроса таблиц
    query = f'''
            SELECT *
            FROM {table}
            LIMIT 5
            '''
    return pd.DataFrame(engine.connect().execute(text(query)))

In [4]:
for tab in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Таблица', tab)
    display(result(tab))

Таблица books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 2. SQL-запросы для решения заданий 

Создадим функцию для выведения результатов запросов

In [5]:
def result(query): 
    return pd.DataFrame(engine.connect().execute(text(query)))

### 2.1 Определение количества вышедших книг после 1 января 2000 года <a id="cell2"></a>  <font size = 2>[к оглавлению](#cell0)</font>

In [6]:
query = '''
            SELECT count(book_id) as count_books
            FROM books
            WHERE publication_date > '2000-01-01'
            ''' 
result(query)

,count_books
0,819


После 1 января 2000 года было опубликовано 819 книг.

### 2.2 Определение количества обзоров и средней оценки для каждой книги <a id="cell3"></a>  <font size = 2>[к оглавлению](#cell0)</font>

In [7]:
query = '''
        SELECT title,
               COUNT(DISTINCT review_id) as reviews_count,
               ROUND(AVG(rating), 2) as rating_avg
        FROM books b
        LEFT JOIN ratings ra ON b.book_id=ra.book_id
        LEFT JOIN reviews re ON b.book_id=re.book_id
        GROUP BY b.book_id
        ORDER BY rating_avg DESC,
                 reviews_count DESC
            ''' 
result(query)

,title,reviews_count,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Всего в исследовании участвуют 1000 книг, которые отсортированы по количеству обзоров пользователей и среднему рейтингу. Самой популярной книгой является "A Dirty Job". Наименьший рейтинг имеет книга "Harvesting the Heart".

### 2.3 Определение издательства, которое выпустило наибольшее число книг толще 50 страниц <a id="cell4"></a>  <font size = 2>[к оглавлению](#cell0)</font>

In [8]:
query = '''
            SELECT p.publisher,
                   COUNT(b.book_id) as count_max 
            FROM books b
            LEFT JOIN publishers p ON p.publisher_id = b.publisher_id 
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY count_max DESC
            LIMIT 1
            ''' 
result(query)

,publisher,count_max
0,Penguin Books,42


Издательство Penguin Books выпустило наибольшее число уникальных изданий, не учитывая брошюры.

### 2.4 Определение автора с самой высокой средней оценкой книг — учитываются только книги с 50 и более оценками <a id="cell5"></a>  <font size = 2>[к оглавлению](#cell0)</font>

In [9]:
# Сначала найдем книги, которые имеют 50 более оценок,
# затем определим автора с самой высокой средней оценкой книг
query = ''' WITH rat_2 AS
            (SELECT book_id,
                    COUNT(rating) as count_rating
            FROM ratings
            GROUP BY book_id
            HAVING COUNT(rating) >= 50
            ORDER BY count_rating)
            
            SELECT a.author,
                   ROUND(AVG(r.rating), 2) as avg_rating
            FROM rat_2
            INNER JOIN books b ON rat_2.book_id = b.book_id
            INNER JOIN authors a ON a.author_id = b.author_id
            INNER JOIN ratings r ON r.book_id = b.book_id
            GROUP BY a.author
            ORDER BY AVG(r.rating) DESC
            LIMIT 1
            ''' 
           
result(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.29


Самый высокий рейтинг книг с количеством оценок 50 и более имеет автор J.K. Rowling/Mary GrandPré (4,29).

### 2.5 Расчет среднего количества обзоров от пользователей, которые поставили больше 50 оценок. <a id="cell6"></a>  <font size = 2>[к оглавлению](#cell0)</font>

In [10]:
# Сначала найдем пользователей, которые поставили более 50 оценок, и посчитаем для них количество обзоров,
# затем определим среднее по обзорам
query = ''' WITH tab_1 AS 
            (SELECT rat.username,
                   COUNT(DISTINCT rat.rating_id) as count_rating,
                   COUNT(DISTINCT rev.review_id) as count_review
            FROM ratings rat
            JOIN reviews rev ON rat.username = rev.username
            GROUP BY rat.username
            HAVING COUNT(DISTINCT rat.rating_id) > 50)
            
            SELECT ROUND(AVG(count_review), 2) as avg_review
            FROM tab_1
            ''' 
           
result(query)

,avg_review
0,24.33


Среди пользователей, которые поставили более 50 оценок, среднее количество обзоров книг составляет 24,33.

## 3. Общий вывод <a id="cell7"></a>  <font size = 2>[к оглавлению](#cell0)</font>

В рамках работы были исследованы таблицы - выведены первые строки. 

Сделаны SQL-запросы для решения вопросов:

   -	Cколько книг вышло после 1 января 2000 года;     
      #### После 1 января 2000 года было выпущено 819 книг.


   -	Для каждой книги посчитайте количество обзоров и среднюю оценку;
#### Всего в исследовании участвуют 1000 книг, которые отсортированы по количеству обзоров пользователей и среднему рейтингу. Самой популярной книгой является "A Dirty Job". Наименьший рейтинг имеет книга "Harvesting the Heart".


   -	Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
#### Издательство Penguin Books выпустило наибольшее число уникальных изданий, не учитывая брошюры.


   -	Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
#### Самый высокий рейтинг книг с количеством оценок 50 и более имеет автор J.K. Rowling/Mary GrandPré (4,29)


   -	Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
#### Среди пользователей, которые поставили более 50 оценок, среднее количество обзоров книг составляет 24,33.